In [1]:
import pandas as pd

In [2]:
asset_vulnerability_df = pd.read_csv(r'C:\Users\swaroop.srisailam\Desktop\Swaroop\Bayesian Risk Assessment using Cybersecurity Data\dataset\asset_vulnerability.csv')

In [3]:
asset_vulnerability_df

,Asset,Vulnerability,CVSS Score,Exploit Probability
0,Server-01,CVE-2023-12345,9.0,0.75
1,Workstation-01,CVE-2023-54321,7.5,0.55
2,Database-01,CVE-2023-67890,8.5,0.70
3,WebApp-01,CVE-2023-09876,9.1,0.68
4,IoTDevice-01,CVE-2023-67891,6.8,0.45
